<a href="https://colab.research.google.com/github/camilaafb/camilaafb/blob/main/Parque_Tecnologico_SJC_vagas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install requests beautifulsoup4 pandas

In [2]:
# Instalar Selenium
!pip install selenium

# Instalar o ChromeDriver (para Chromium) e o Chromium
!apt-get update # Atualiza a lista de pacotes
!apt-get install chromium-browser # Instala o navegador Chromium
!apt-get install chromium-chromedriver # Instala o driver do Chromium

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:4 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading

In [3]:
!pip install google-colab-selenium

In [67]:
import pandas as pd
import sqlite3
import time # Para adicionar delays
import google_colab_selenium as gs
from selenium import webdriver
from selenium.webdriver.common.by import By # Para localizar elementos
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import WebDriverException, NoSuchElementException, TimeoutException
from selenium.webdriver.support import expected_conditions as EC

In [57]:
driver = gs.Chrome()

<IPython.core.display.Javascript object>

In [58]:
url_base = 'https://pitsjc.org.br/empresas/'

In [59]:
driver.get(url_base)
print(driver.title)

Empresas - Parque de Inovação Tecnológica São José dos Campos


In [68]:
# Espera explícita inicial para o primeiro lote de empresas carregar
try:
    WebDriverWait(driver, 15).until(
        EC.presence_of_all_elements_located((By.CLASS_NAME, 'search-filter-ficha'))
    )
    print("Primeiro lote de fichas de empresa carregado.")
except TimeoutException:
    print("Tempo limite excedido. Nenhuma ficha de empresa encontrada na carga inicial. Verifique a URL ou o seletor.")
    driver.quit()
    exit()

Primeiro lote de fichas de empresa carregado.


In [69]:
# --- Lógica de Rolagem Infinita ---
# A ideia é rolar a página e coletar novas empresas até que nenhuma nova empresa seja carregada
last_height = driver.execute_script("return document.body.scrollHeight")
scroll_attempts = 0
max_scroll_attempts = 100 # Limite para evitar loop infinito em caso de erro

while True and scroll_attempts < max_scroll_attempts:
    # Rola para o final da página
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Espera um pouco para o novo conteúdo carregar.
    # O tempo pode precisar de ajuste dependendo da velocidade do site.
    time.sleep(3)

    # Calcula a nova altura da página de rolagem e compara com a última altura
    new_height = driver.execute_script("return document.body.scrollHeight")

    if new_height == last_height:
        # Se a altura não mudou, significa que não há mais conteúdo para carregar
        print("Fim da página alcançado ou nenhum novo conteúdo carregado.")
        break
    else:
        last_height = new_height
        scroll_attempts += 1
        print(f"Página rolada. Nova altura: {new_height}. Tentativa de rolagem: {scroll_attempts}")

Página rolada. Nova altura: 11398. Tentativa de rolagem: 1
Página rolada. Nova altura: 19884. Tentativa de rolagem: 2
Página rolada. Nova altura: 28376. Tentativa de rolagem: 3
Página rolada. Nova altura: 36874. Tentativa de rolagem: 4
Página rolada. Nova altura: 45378. Tentativa de rolagem: 5
Página rolada. Nova altura: 53870. Tentativa de rolagem: 6
Página rolada. Nova altura: 62368. Tentativa de rolagem: 7
Página rolada. Nova altura: 70872. Tentativa de rolagem: 8
Página rolada. Nova altura: 79364. Tentativa de rolagem: 9
Página rolada. Nova altura: 87856. Tentativa de rolagem: 10
Página rolada. Nova altura: 92126. Tentativa de rolagem: 11
Página rolada. Nova altura: 96090. Tentativa de rolagem: 12
Fim da página alcançado ou nenhum novo conteúdo carregado.


In [74]:
# --- 3. Extrair os dados de TODAS as fichas de empresa após rolar tudo ---
# Agora que toda a página (ou o máximo possível) foi carregada, você pode raspar.
# Isso é mais eficiente do que raspar a cada rolagem, pois evita duplicatas e
# acessa o DOM uma única vez para todos os elementos.

lista_empresas = []

fichas_empresas = driver.find_elements(By.CLASS_NAME, 'search-filter-ficha')
print(f"Total de fichas de empresa encontradas: {len(fichas_empresas)}")

for ficha_elem in fichas_empresas:
  # Extrair nome da empresa
  try:
      nome_link_elem = ficha_elem.find_element(By.XPATH, ".//div[@class='search-filter-info-empresa']/h4/a")
      nome_empresa = nome_link_elem.text.strip()
  except NoSuchElementException:
      # Skip this entry if the company name element is not found (unlikely, but good practice)
      print("Warning: Could not find company name for an entry.")
      continue
  # Extrair link do site da empresa
  link_website = None # Inicializa com None para o caso de não encontrar o link
  try:
      link_site_elem = ficha_elem.find_element(By.XPATH, ".//i[@class='fas fa-link']/following-sibling::a")
      link_website = link_site_elem.get_attribute('href')
  except NoSuchElementException:
      # Se o link do site não for encontrado, link_website permanece None
      print(f"Warning: Could not find website link for company: {nome_empresa}. Setting link to None.")
      pass # Continua a execução mesmo se o link não for encontrado
  # Extrair o bloco de informações da empresa para processar a localização
  info_div = ficha_elem.find_element(By.CLASS_NAME, 'search-filter-info-empresa')
  full_info_text = info_div.text
  lines = full_info_text.split('\n')
# Localiza o texto do nome da empresa e o link do site (se presente)
  nome_da_empresa_no_texto = nome_empresa
  link_do_site_no_texto = link_website

  endereco_lines = []
  capturing_address = False
  for line in lines:
      if nome_da_empresa_no_texto in line and not capturing_address:
          capturing_address = True # Começa a capturar após o nome da empresa
          continue
      if capturing_address:
          # Verifica se a linha contém um número de telefone ou um link
          if '(' in line and ')' in line and (line.replace('(', '').replace(')', '').replace('-', '').replace(' ', '').isdigit() or "phone" in line.lower()): # Heurística simples para telefone
              break # Chegou no telefone
          if 'http://' in line or 'https://' in line:
              break # Chegou no link
          if 'CEP' in line and line.replace('CEP', '').replace(' ', '').replace('-', '').isdigit():
              endereco_lines.append(line.strip())
              break # O CEP geralmente é a última linha do endereço

          endereco_lines.append(line.strip())

  localizacao_empresa = ' '.join(endereco_lines).strip()
  # Limpeza adicional para remover quebras de linha e múltiplos espaços
  localizacao_empresa = ' '.join(localizacao_empresa.split())
  lista_empresas.append({
      'Nome da Empresa': nome_empresa,
      'Link do Website': link_website,
      'Localização': localizacao_empresa
      })




Total de fichas de empresa encontradas: 439


In [76]:
driver.quit()
print("\nNavegação concluída e WebDriver encerrado.")


Navegação concluída e WebDriver encerrado.


In [77]:
df = pd.DataFrame(lista_empresas)

In [79]:
df_sjc = df[df['Localização'].str.contains('São José dos Campos', na=False, case=False)].copy()

In [80]:
df_sjc.to_csv('empresas_com_localizacao_sjc.csv', index=False, encoding='utf-8')
print("\nDados salvos em empresas_com_localizacao_sjc.csv")


Dados salvos em empresas_com_localizacao_sjc.csv


In [ ]:


df.to_csv('empresas_com_localizacao.csv', index=False, encoding='utf-8')
print("\nDados salvos em empresas_com_localizacao.csv")

conn = sqlite3.connect('empresas_com_localizacao.db')
df.to_sql('empresas', conn, if_exists='replace', index=False)
conn.close()
print("Dados salvos em empresas_com_localizacao.db (SQLite)")

print("\nPrimeiras 5 linhas do DataFrame:")
print(df.head())